# ETL IV Pair 24/08


In [3]:
import pandas as pd
import mysql.connector

## ETL Carga I

1. Cread la BBDD con el nombre de energía. La BBDD tendrá la siguiente forma:
2. Cread las tablas de la BBDD:
- Tabla fechas
- Tabla nacional_renovable_no_renovable
- Tabla comunidades_renovable_no_renovable
- Tabla comunidades
Al final nuestra BBDD tendrá el siguiente aspecto:
    
3. BONUS Insertar los datos en las tablas.
📌 Nota Esta parte del pair es optativa y no será considerada para la evaluación

In [ ]:
df_energia_ccaa = pd.read_pickle("data/pairETL1_datos_energia_ccaa.pkl")
df_energia_ccaa.head()

In [ ]:
def crear_bbdd(nombre_bbdd):

    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="AlumnaAdalab", 
      auth_plugin = 'mysql_native_password') 
    print("Conexión realizada con éxito")
    
    mycursor = mydb.cursor()

    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [ ]:
mydb = crear_bbdd("energía")

In [ ]:
def crear_insertar_tabla(nombre_bbdd, contraseña, query):
 
    cnx = mysql.connector.connect(user='root', password=f"{contraseña}",
                                     host='127.0.0.1', database=f"{nombre_bbdd}",  
                                     auth_plugin = 'mysql_native_password')

    mycursor = cnx.cursor()
    
   
    try: 
        mycursor.execute(query)
        cnx.commit() 
   
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [ ]:
tabla_fechas = '''
CREATE TABLE IF NOT EXISTS `clima`.`tiempo` (
  `idtiempo` INT NOT NULL AUTO_INCREMENT,
  `timepoint`INT NOT NULL,
  `fecha` DATE NOT NULL,
  PRIMARY KEY (`idtiempo`))
ENGINE = InnoDB;
'''

In [ ]:
tabla_localidades = '''
CREATE TABLE IF NOT EXISTS `clima`.`localidades` (
  `idlocalidad` INT NOT NULL AUTO_INCREMENT,
  `latitud` DECIMAL(6,4) NOT NULL,
  `longitud` DECIMAL(6,4) NOT NULL,
  `ciudad` VARCHAR(45) NOT NULL,
  PRIMARY KEY (`idlocalidad`))
ENGINE = InnoDB;
'''